<a href="https://colab.research.google.com/github/Velociraptorvelraptor/pytorch-lightening-intel-img-classification/blob/main/intel_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle -q

In [ ]:
!mkdir /root/.kaggle 

In [ ]:
!mv /content/kaggle.json /root/.kaggle/kaggle.json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!kaggle datasets download -d puneet6060/intel-image-classification

In [ ]:
!mv /content/intel-image-classification.zip '/content/drive/MyDrive/Colab Notebooks/intel-image-classification'

In [ ]:
!unzip '/content/drive/MyDrive/Colab Notebooks/intel-image-classification/intel-image-classification.zip' -d '/content/drive/MyDrive/Colab Notebooks/intel-image-classification/data'



In [ ]:
src_path = '/content/drive/MyDrive/Colab Notebooks/intel-image-classification'

In [ ]:
train_path = src_path + '/data/seg_train/seg_train/'
test_path = src_path + '/data/seg_test/seg_test/'

In [ ]:
train_path

In [ ]:
!pip install pytorch_lightning==1.8.5 torchvision torchmetrics

In [ ]:
import os
from types import SimpleNamespace

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import random_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
from torchmetrics.classification import MulticlassAccuracy
import torchvision
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torchmetrics import ConfusionMatrix, F1Score, ROC


In [ ]:
transform = T.Compose([T.ToTensor(), 
                       T.Resize((64, 64)),
                       T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
train_dataset = ImageFolder(train_path, transform)
test_dataset = ImageFolder(test_path, transform)

In [ ]:
train_dataset

In [ ]:
img, label = train_dataset[3]; img.shape

In [ ]:
def img_display(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    npimg = np.transpose(npimg, (1, 2, 0))
    return npimg

In [ ]:
train_idx = int(len(train_dataset) * 0.8)
val_idx = len(train_dataset) - train_idx

In [ ]:
train_dataset, val_dataset = random_split(train_dataset, [train_idx, val_idx])

In [ ]:
sample_img, sample_label = val_dataset.__getitem__(0)

plt.imshow(img_display(sample_img))
plt.title(f'Encoded label: {sample_label}')

In [ ]:
BATCH_SIZE = 64

In [ ]:
train_dl = DataLoader(train_dataset, BATCH_SIZE, shuffle=True)
val_dl = DataLoader(val_dataset, BATCH_SIZE, shuffle=True)
test_dl = DataLoader(test_dataset, BATCH_SIZE, shuffle=True)

In [ ]:
def _conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
              nn.BatchNorm2d(out_channels), 
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class LitModel(pl.LightningModule):
    def __init__(self, in_channels=3, num_classes=6, learning_rate=1e-3):
        super().__init__()
        self.learning_rate = learning_rate
        self.num_classes = num_classes
        self.train_accuracy = MulticlassAccuracy(self.num_classes)
        self.val_accuracy = MulticlassAccuracy(self.num_classes)
        self.test_accuracy = MulticlassAccuracy(self.num_classes)
        self.confmat = ConfusionMatrix(task="multiclass", num_classes=self.num_classes)
        self.f1 = F1Score(task="multiclass", num_classes=self.num_classes)
        self.roc = ROC(task="multiclass", num_classes=self.num_classes)

                # Define PyTorch model
        self.conv1 = _conv_block(in_channels, 64)
        self.conv2 = _conv_block(64, 128, pool=True)
        self.res1 = nn.Sequential(_conv_block(128, 128), _conv_block(128, 128))
        
        self.conv3 = _conv_block(128, 256, pool=True)
        self.conv4 = _conv_block(256, 512, pool=True)
        self.res2 = nn.Sequential(_conv_block(512, 512), _conv_block(512, 512))
        
        self.classifier = nn.Sequential(nn.AdaptiveMaxPool2d(1), 
                                        nn.Flatten(), 
                                        nn.Dropout(0.2),
                                        nn.Linear(512, self.num_classes))

    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.train_accuracy.update(preds, y)
        self.log("train_loss", loss, prog_bar=True, on_epoch=True)
        self.log("train_acc", self.train_accuracy, prog_bar=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.val_accuracy.update(preds, y)
        self.log("val_loss", loss, prog_bar=True, on_epoch=True)
        self.log("val_acc", self.val_accuracy, prog_bar=True, on_epoch=True)

    def test_step(self, batch, batch_idx):
        acc_list = []
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.test_accuracy.update(preds, y)
        self.confmat.update(preds, y)
        self.f1.update(preds, y)
        # Calling self.log will surface up scalars for you in TensorBoard
        self.log("test_loss", loss, on_epoch=True)
        self.log("test_acc", self.test_accuracy, on_epoch=True)
        self.log("f1", self.f1, on_epoch=True)

    def on_test_end(self):
        cm_tensor = self.confmat.compute()
        fig = plt.figure(figsize=[10,10])
        plt.title(f"Confusion matrix")
        sns.heatmap(cm_tensor.cpu().numpy(), annot=True, cmap='Blues')

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [ ]:
model = LitModel(); model

In [ ]:
trainer = pl.Trainer(auto_lr_find=True, gpus=-1, limit_train_batches=0.1)
lr_finder = trainer.tuner.lr_find(model, train_dataloaders=train_dl)

In [ ]:
# Plot with
fig = lr_finder.plot(suggest=True)
fig.show()

In [ ]:
new_lr = lr_finder.suggestion()

# update hyperparams of the model
model.learning_rate = new_lr

In [ ]:
checkpoint_callback = ModelCheckpoint(dirpath=src_path + '/checkpoints/', save_top_k=2, monitor="val_loss")

In [ ]:
trainer = pl.Trainer(
    accelerator="auto",
    devices=1 if torch.cuda.is_available() else None,  # limiting got iPython runs
    max_epochs=50,
    logger=CSVLogger(save_dir="logs/"),
    limit_train_batches=0.2,
    limit_val_batches=0.05,
    callbacks=[checkpoint_callback])

In [ ]:
trainer.fit(model, train_dl, val_dl)

In [ ]:
metrics = pd.read_csv(f"{trainer.logger.log_dir}/metrics.csv")
del metrics["step"]
metrics.set_index("epoch", inplace=True)
display(metrics.dropna(axis=1, how="all").head())
sns.relplot(data=metrics, kind="line")

In [ ]:
model = LitModel.load_from_checkpoint(checkpoint_path= src_path + "/checkpoints/epoch=9-step=350.ckpt")

In [ ]:
trainer.test(model, test_dl, ckpt_path=src_path + "/checkpoints/epoch=9-step=350.ckpt")